# Діагностичний аналіз пухлини молочної залози

## 1. Визначення проблеми
Рак молочної залози (РМЗ) – одне з найчастіших онкологічнихзахворювань у світі серед жінок з високою захворюваністю та смертністю. Воно становить 23-27% від злоякісних новоутворень, які зустрічаються у жінок, і розвивається в два рази частіше, ніж рак іншої локалізації. Рак молочної залози виникає як результат аномального росту клітин тканини молочної залози, що зазвичай називають пухлиною. Пухлина не означає рак - пухлина може бути доброякісною (не раковою), перед злоякісною (передраковою) або злоякісною (раковою). Такі діагнози, як МРТ, мамографія, ультразвукове дослідження та біопсія, зазвичай використовуються для діагностики раку молочної залози.

### 1.1. Очікуваний результат
Вхідні дані - повно-слайдові зображення морфології тканини молочної залози, отриманої шляхом тонкоголкової аспіраційної пункційної біопсії (ТАПБ) (це діагностична процедура, суть якої полягає у отриманні клітин з підозрілого новоутворення для його діагностування). Для аналізу цих зображень необхідно побудувати модель, яка може класифікувати пухлини молочної залози, використовуючи дві класифікації навчання:
* 1 = Злоякісна (Ракова) - Наявна
* 0 = Доброякісна (Не ракова) - Відсутня

### 1.2. Мета
Оскільки мітки в даних дискретні, передбачення поділяється на дві категорії (злоякісні/доброякісні). У машинному навчанні це проблема класифікації. Таким чином, мета полягає в класифікації доброякісних та злоякісних утворень молочної залози, і в прогнозуванні рецидиву та повторного виникнення злоякісних утворень через певний період часу. Для цього ми використовували методи класифікації машинного навчання, у відповідності з функцією, яка може передбачити дискретні класи нових вхідних даних.

### 1.3. Джерела даних
Набори даних раку молочної залози для машинного навчання, які було використано в даній роботі, знаходяться у відкритому доступі в [репозиторії](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29) університета Каліфорнії, Ірвайн. Набір даних містить **569 зразків злоякісних і доброякісних пухлин клітин молочної залози**.
* У перших двох стовпцях набору даних зберігаються унікальні ідентифікаційні номери зразків та відповідний діагноз (M (Malignant) = злоякісний, B (Benign) = доброякісний) відповідно.
* Стовпці 3-32 містять 30 реальних ознак, обчислених з оцифрованих зображень ядер клітин, які можна використовувати для побудови моделі для прогнозування доброякісної чи злоякісної пухлини.

#### Початок роботи: підключення необхідних бібліотек та встановлення параметрів

In [ ]:
#підключення бібліотек
import numpy as np         # лінійна алгебра
import pandas as pd        # обробка даних, CSV file I/O (e.g. pd.read_csv)

# Читання файлу "data.csv" та виведення його вмісту.
#!cat data/data.csv
data = pd.read_csv('data/data.csv', index_col=False,)

#### Завантаження набору даних
Спочатку необхідно завантажити CSV-файл, використовуючи функцію Pandas read_csv().

#### Перевірка даних
Перший крок - візуально перевірити новий набір даних. Існує кілька способів це зробити:
* Найпростіше запросити перші кілька записів методом DataFrame **data.head()**. За замовчуванням **data.head()** повертає перші 5 рядків з DataFrame об’єкта df (виключаючи рядок заголовка).
* Крім того, можна також використовувати **df.tail()** для повернення п'яти рядків даних.
* Для методів як **data.head()**, так і для **df.tail()** є можливість вказати кількість записів, включивши необхідне число між дужками при виклику будь-якого методу.

In [ ]:
data.head(2)

Можна відфільтрувати вибірку, використовуючи метод **data.drop()**, як показано нижче.

In [ ]:
# Стовпець id зайвий на даній вибірці, тому його можна опустити
data.drop('id', axis =1, inplace=True)
#data.drop('Unnamed: 0', axis=1, inplace=True)
data.head(2)

In [ ]:
data.shape

Результатом вибірки є 569 записів, кожен з яких має 32 стовпці.

Метод **info ()** надає стислий підсумок даних; надає інформацію про тип даних, кількість ненульових значень у кожному стовпчику та кількість пам'яті, яку використовує запис.

Метод **get_dtype_counts()** поверне кількість стовпців кожного DataFrame типу:

In [ ]:
# Перегляд інформації про типи даних за допомогою data.info()
data.info()

In [ ]:
# Кількість стовпців кожного типу даних у DataFrame
data.get_dtype_counts()

З наведених вище результатів із 32 змінних, id стовпця - це ціле число, діагнози - not null, а решта - float.

In [ ]:
# перевірка відсутніх даних
#data.isnull().any()

In [ ]:
data.diagnosis.unique()

З наведених вище результатів видно, що **діагноз** є категорійною змінною, оскільки вона представляє фіксовану кількість можливих значень (злоякісна/доброякісна). Алгоритми машинного навчання вимагають числові значення, а не рядки, як вхідні дані, тому потрібен певний метод їх для їх конвертації.

In [ ]:
# збереження даних у відфільтрованому вигляді для подальшого аналізу
data.to_csv('data/clean-data.csv')